In [10]:
import math
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import random
import statistics


from SALib.sample import saltelli
from SALib.analyze import sobol

# Aquifer thickness b [m]
b = 10

# Hydraulic Conductivity from Cobuto, 1999 (adapted): 10^-4 - 10^-2 [m/s]
Kmin = 1e-04 * 86400
Kmax = 1e-02 * 86400

# Specific yield Sy from Johnson, 1967: 0.21 - 0.35 [-]
Sy_min = 0.21
Sy_max = 0.35

# Calculating D using eq. (2) from Sawyer et al., 2014
Dmin_temp = (Kmin*b)/Sy_max
Dmax_temp = (Kmax*b)/Sy_min

Dmin = math.log(Dmin_temp, 10) # lower bound for diffusivity
Dmax = math.log(Dmax_temp, 10) # upper bound for diffusivity
phi = 0 # phase

def solution(x, t, A, omega, D, phi):
  """This is the function which gives back the hydraulic head
  omega: frequency
  A: amplitude
  x: distance
  t: time"""
  sinus = math.sin(-x*math.sqrt(omega/(2*D))+ omega*t + phi)
  ret = A*math.exp(-x*math.sqrt(omega/(2*D)))*sinus
  return ret 




In [11]:
class Result:
   def __repr__(self):
    return 'D={}, A={}, w={}, x={}, t={}, hxt={}'.format(
      self.D,
      self.A,
      self.w,
      self.x,
      self.t,
      self.hxt,        
    )

def calculate_hxt_for_one_variable(D_input, A_input, w_input, x_input, t_input):
  fun1 = A_input * math.exp(-x_input * math.sqrt(w_input / (2*D_input)))
  fun2 = math.sin(-x_input * math.sqrt((w_input / (2*D_input)) + ((w_input * t_input) )))
  return fun1 * fun2

In [12]:
# Define a general function to generate n different realizations of the hydraulic head.
def calculate_hxt(x_vector, t_vector, A_vector, w_vector, D_vector):
	results = []

	for simulation_index in range(0, TOTAL_NUMBER_OF_SIMULATIONS):
		temp_result = Result()

		for x_index in range(0, len(x_vector)):
			for t_index in range(0, len(t_vector)):
				temp_result = Result()				
				temp_result.D = D_vector[simulation_index]
				temp_result.A = A_vector[simulation_index]
				temp_result.w = w_vector[simulation_index]
				temp_result.x = x_vector[x_index]
				temp_result.t = t_vector[t_index]
				temp_result.hxt = solution(temp_result.x, temp_result.t,temp_result.A, temp_result.w,  temp_result.D, phi)
				#temp_result.hxt = calculate_hxt_for_one_variable(temp_result.D, temp_result.A, temp_result.w, temp_result.x, temp_result.t)
				results.append(temp_result.hxt)

	return results

In [13]:
num_of_simulations = 10000

# vectors with 10000 samples of the parameters in matrix A & B
D_vector_A = [10**((Dmax - Dmin)*random.random() + Dmin) for _ in range(num_of_simulations)]
A_vector_A = [10**(random.random() - 1) for _ in range(num_of_simulations)]
omega_vector_A = [ (math.pi - 2*math.pi/7)*random.random() + 2*math.pi/7 for _ in range(num_of_simulations)]



D_vector_B = [10**((Dmax - Dmin)*random.random() + Dmin) for _ in range(num_of_simulations)]
A_vector_B = [10**(random.random() - 1) for _ in range(num_of_simulations)]
omega_vector_B = [ (math.pi - 2*math.pi/7)*random.random() + 2*math.pi/7 for _ in range(num_of_simulations)]

In [14]:
def calculate_solution(x, t, A_vector, omega_vector, D_vector):
    # return 10000 simulations of the hydraulic head
    y = []

    for idx in range(num_of_simulations):
        hydraulic_head = solution(x, t, A_vector[idx], omega_vector[idx], D_vector[idx], phi)
        y.append(hydraulic_head)
  
    return y

In [15]:
distances = [1, 10, 100] # given in meters
timesteps = [7, 30, 180] # given in days


TOTAL_NUMBER_OF_SIMULATIONS = 10000

for x in distances:
    for t in timesteps:

        y_A = calculate_solution(x, t, A_vector_A, omega_vector_A, D_vector_A)
        y_B = calculate_solution(x, t, A_vector_B, omega_vector_B, D_vector_B)
        y_C_1 = calculate_solution(x, t, A_vector_A, omega_vector_B, D_vector_B)
        y_C_2 = calculate_solution(x, t, A_vector_B, omega_vector_A, D_vector_B)
        y_C_3 = calculate_solution(x, t, A_vector_B, omega_vector_B, D_vector_A)


        avg = (sum(y_A)/num_of_simulations) * (sum(y_B)/num_of_simulations)
        print(avg)
        
        # First order sensitivity indices
        
        S_1 = ((sum([hyd_head_A*hyd_head_B for hyd_head_A, hyd_head_B in zip(y_A, y_C_1)]) / num_of_simulations) - avg) / ((sum([hyd_head_A ** 2 for hyd_head_A in y_A]) / num_of_simulations) - avg)
        S_2 = ((sum([hyd_head_A*hyd_head_B for hyd_head_A, hyd_head_B in zip(y_A, y_C_2)]) / num_of_simulations) - avg) / ((sum([hyd_head_A ** 2 for hyd_head_A in y_A]) / num_of_simulations) - avg)
        S_3 = ((sum([hyd_head_A*hyd_head_B for hyd_head_A, hyd_head_B in zip(y_A, y_C_3)]) / num_of_simulations) - avg) / ((sum([hyd_head_A ** 2 for hyd_head_A in y_A]) / num_of_simulations) - avg)

        print('S_A='+str(S_1))
        print('S_omega='+str(S_2))
        print('S_D='+str(S_3))
        print(" ")
        
        # Total order sensitivity indices
        
        ST_1 = 1 - (((sum([hyd_head_A*hyd_head_B for hyd_head_A, hyd_head_B in zip(y_B, y_C_1)]) / num_of_simulations) - avg) / ((sum([hyd_head_A ** 2 for hyd_head_A in y_A]) / num_of_simulations) - avg))
        ST_2 = 1 - (((sum([hyd_head_A*hyd_head_B for hyd_head_A, hyd_head_B in zip(y_B, y_C_2)]) / num_of_simulations) - avg) / ((sum([hyd_head_A ** 2 for hyd_head_A in y_A]) / num_of_simulations) - avg))
        ST_3 = 1 - (((sum([hyd_head_A*hyd_head_B for hyd_head_A, hyd_head_B in zip(y_B, y_C_3)]) / num_of_simulations) - avg) / ((sum([hyd_head_A ** 2 for hyd_head_A in y_A]) / num_of_simulations) - avg))

        print('ST_A='+str(ST_1))
        print('ST_omega='+str(ST_2))
        print('ST_D='+str(ST_3))
        print(" ")

0.0021892208246004114
S_A=0.005470404091043931
S_omega=0.6981320163556537
S_D=0.0001806313341988213
 
ST_A=0.32088622636045727
ST_omega=0.9930234444648388
ST_D=0.034523950300849204
 
2.1354808187170676e-05
S_A=-0.012277084717563105
S_omega=0.7054037667273136
S_D=-0.001385785227457967
 
ST_A=0.30452844393625855
ST_omega=0.9783121200889878
ST_D=0.009926339855548694
 
4.064382128184194e-06
S_A=0.017327427768448814
S_omega=0.7102485465041579
S_D=0.00020752269241528044
 
ST_A=0.2997383043370937
ST_omega=1.0087820940150871
ST_D=0.025580189247392693
 
0.0014349396825741741
S_A=0.014292565726165553
S_omega=0.669136464762918
S_D=0.0029200562290400043
 
ST_A=0.32449588483104674
ST_omega=0.984504734423765
ST_D=0.07340189458077007
 
5.402335515654622e-06
S_A=-0.00852706794746026
S_omega=0.6767858113540322
S_D=-0.00371921634421258
 
ST_A=0.3075558563121128
ST_omega=0.9663564704547449
ST_D=0.05414159915982564
 
6.404800200408856e-07
S_A=0.023545243983339247
S_omega=0.6825101380092496
S_D=0.002572628

In [16]:
inpt = {
    'num_vars': 3,
    'names': ['A', 'w', 'D'],
    'bounds': [[10**(-1) , 1],
               [(2*math.pi)/7, (2*math.pi)/2],
               [10**Dmin, 10**Dmax]],
    'dists': ['unif', 'unif', 'unif'] 
}

param_values = saltelli.sample(inpt, N = 2**13, calc_second_order=True)

C:\Users\z004n49n\AppData\Local\Temp\ipykernel_6952\4211220531.py:10: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  param_values = saltelli.sample(inpt, N = 2**13, calc_second_order=True)


In [17]:
def return_hydraulic_head(values, x_input, t_input):
    Y = np.zeros([values.shape[0]])
    for i,X in enumerate(values):
          Y[i] = (X[0] * math.exp(-x_input * math.sqrt(X[1] / (2*X[2])))) * (math.sin(-x_input * math.sqrt(X[1] / (2*X[2])) + ((X[1] * t_input))))  
    return Y

In [18]:
for x in distances:
    for t in timesteps:
        
        Y = return_hydraulic_head(param_values, x_input = np.array([x]), t_input = np.array([t]))
        sobol.analyze(inpt, Y, print_to_console=True)

         ST   ST_conf
A  0.187392  0.006573
w  0.994485  0.027732
D  0.000079  0.000009
         S1   S1_conf
A  0.006209  0.014484
w  0.813115  0.025314
D -0.000004  0.000218
              S2   S2_conf
(A, w)  0.181223  0.021092
(A, D) -0.000002  0.024342
(w, D)  0.000039  0.023978
         ST   ST_conf
A  0.182562  0.006313
w  1.003610  0.029661
D  0.000084  0.000010
         S1   S1_conf
A -0.005315  0.014234
w  0.826339  0.026159
D -0.000173  0.000263
              S2   S2_conf
(A, w)  0.187888  0.022761
(A, D)  0.010564  0.025884
(w, D)  0.000102  0.021130
         ST   ST_conf
A  0.182512  0.005502
w  0.948832  0.030467
D  0.000077  0.000009
         S1   S1_conf
A  0.009402  0.013544
w  0.774192  0.024791
D -0.000072  0.000301
              S2   S2_conf
(A, w)  0.173808  0.021790
(A, D) -0.000055  0.024583
(w, D)  0.002118  0.021805
         ST   ST_conf
A  0.187426  0.006696
w  0.994044  0.038461
D  0.006445  0.000574
         S1   S1_conf
A  0.006210  0.013889
w  0.807621  0.0